In [1]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s

mp4 to mp3

In [2]:
!pip install ffmpeg-python
import ffmpeg

def mp4tomp3(video_file_path, mp3_file_path):
    try:
        (
            ffmpeg
            .input(video_file_path)
            .output(mp3_file_path, format="mp3", audio_bitrate="192k")
            .run(overwrite_output=True)
        )
        print(f"Conversion successful: {mp3_file_path}")
    except Exception as e:
        print(f"Error: {e}")


In [3]:
mp4tomp3('meet.mp4','meetaudio.mp3')

Conversion successful: meetaudio.mp3


transcribing

In [6]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe("meetaudio.mp3",language='en')
print(result["text"])


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

 Good morning, everyone. I'm Arushi Kumari, and I will be talking about ethics in business, featuring technology in the current age. The current day and age seeing a lot of very quick and life-altering developments in all aspects of life. Like it or not, our lives are deeply and generously guided by businesses and every action they take deeply influences our lives. You will think how does a product introduced by a business affect us? Look around you. Every single thing you use has been chosen and selected for you. For something so intricately interwoven into our day-to-day lives, a guiding principle for its functioning becomes important. In the realm of business, ethics serve as a guiding principle and shapes all decision-making processes and behaviors. And we are here to discuss just that, ethics in business. Businesses usually follow two ethical perspectives, deontological and utilitarian perspective. Deontological ethics is rooted in the principles of duty and moral obligation. It e

In [7]:
transcript = result["text"]

translating transcript (if needed)


In [ ]:
from googletrans import Translator

def detect_language(text):
    translator = Translator()
    detected_lang = translator.detect(text).lang  # Detects language
    return detected_lang

def translate_transcript(text, target_lang='en'):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_lang).text
    return translated_text

querying transcript

In [1]:
!pip install faiss-cpu==1.8.0 unstructured==0.14.5 unstructured[pdf]==0.14.5  transformers==4.41.2 sentence-transformers==3.0.1
!pip install langchain-community
!pip install -q groq
!pip install langchain-groq
!pip install groq langchain_groq
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [2]:
import os
from groq import Groq
from langchain_groq import ChatGroq
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

from getpass import getpass
api_key = getpass('Enter your API key: ')
os.environ["GROQ_API_KEY"] = api_key

Enter your API key: ··········


In [8]:
def gen_mom(transcript):
  llm = ChatGroq(
    model = "llama-3.1-8b-instant",
    temperature=0
  )
  prompt = f'''You are given the transcript of a meeting recording, being the helpful AI agent you are,
               give a properly structred Minutes of Meetings (MoM) only based on the transcript.
               Minutes of Meetings (MoM) are the official records that record the most important points discussed during a meeting.
               They contain the main discussions, decisions, resolutions, action items, and the attendees.
               MoM is a formal record that keeps the participants updated and ensures accountability for the assigned tasks.

               Contents of MoM:
                A well-read MoM must usually contain the following:

                Date and Time: The date and time that the meeting has taken place should be stated
                Participants: Attendees names and those not present should be indicated.
                Agenda: All topics considered in the meeting should be reflected.
                Key Discussions: Sum up the key matters and deliberation.
                Decisions and Resolutions: Outcomes of each agenda item should be captured.
                Action Items: Activities assigned, individually responsible, and due dates.
                Next Meeting Date: If scheduled, mention the tentative date for the next meeting.

                Transcript:{transcript}
               '''
  response = llm.invoke(prompt)
  return response.content.strip()

In [9]:
print(transcript)
mom = gen_mom(transcript)
print(mom)

 Good morning, everyone. I'm Arushi Kumari, and I will be talking about ethics in business, featuring technology in the current age. The current day and age seeing a lot of very quick and life-altering developments in all aspects of life. Like it or not, our lives are deeply and generously guided by businesses and every action they take deeply influences our lives. You will think how does a product introduced by a business affect us? Look around you. Every single thing you use has been chosen and selected for you. For something so intricately interwoven into our day-to-day lives, a guiding principle for its functioning becomes important. In the realm of business, ethics serve as a guiding principle and shapes all decision-making processes and behaviors. And we are here to discuss just that, ethics in business. Businesses usually follow two ethical perspectives, deontological and utilitarian perspective. Deontological ethics is rooted in the principles of duty and moral obligation. It e

In [11]:
with open("transcript.txt", "w", encoding="utf-8") as file:
    file.write(transcript)


In [4]:
import nltk
nltk.download('punkt_tab')

from langchain.document_loaders import UnstructuredFileLoader

loader = UnstructuredFileLoader("transcript.txt")
meet_transcript = loader.load()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
<ipython-input-4-49897636ca5d>:6: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader("transcript.txt")


In [14]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [9]:
# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = text_splitter.split_documents(meet_transcript)

# Embeddings
embeddings = HuggingFaceEmbeddings()
knowledge_base = FAISS.from_documents(text_chunks, embeddings)

#Querying
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGroq(
    model = "llama-3.1-8b-instant",
    temperature=0
  )

prompt = ChatPromptTemplate.from_template(
    """You are an AI assistant specialized in answering questions based on meeting transcripts.
    Use the retrieved context to provide accurate responses. If the answer isn't in the context,
    say 'I don't know'.

    Context:
    {context}

    Question:
    {input}
    """
)


retriever=knowledge_base.as_retriever()
question_answer_chain = create_stuff_documents_chain(llm, prompt,document_variable_name="context")
rag_chain = create_retrieval_chain(retriever, question_answer_chain)



<ipython-input-9-88b4b183dcf4>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [10]:
while True:
    query = input("\nAsk a question about the meeting (or type 'exit' to quit): ")
    if query.lower() == "exit":
        break
    response = rag_chain.invoke({"input": query})
    print("\nAI Response:", response["answer"])


Ask a question about the meeting (or type 'exit' to quit): what is the topic of the meeting?

AI Response: The topic of the meeting is "ethics in business, featuring technology in the current age."

Ask a question about the meeting (or type 'exit' to quit): do they talk about automation?

AI Response: Yes, they do talk about automation. In the context, it is mentioned that "technology is picking up very quickly. We are in a time where self-driving cars are real, automation is smoother than ever, and our lives are riddled with human-like bots." This suggests that automation is a significant aspect of the current technological advancements being discussed.

Ask a question about the meeting (or type 'exit' to quit): quit

AI Response: I don't know.

Ask a question about the meeting (or type 'exit' to quit): exit
